In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create output directory for charts
import os
os.makedirs('outputs', exist_ok=True)
os.makedirs('csvfiles', exist_ok=True)

print("TRADER BEHAVIOR & MARKET SENTIMENT ANALYSIS")

TRADER BEHAVIOR & MARKET SENTIMENT ANALYSIS


In [8]:
import pandas as pd
from pathlib import Path

# Base directory for CSV files (use a raw string to avoid backslash escapes)
base = Path(r"C:\anuj\Tanuj_Tambe\csv_files")

# Load Fear & Greed Index
fear_greed = pd.read_csv(base / "fear_greed_index.csv")
print(f"✓ Fear & Greed Index loaded: {fear_greed.shape} records")

# Load Historical Trading Data
historical_data = pd.read_csv(base / "historical_data.csv")
print(f"✓ Historical Trading Data loaded: {historical_data.shape} records")

✓ Fear & Greed Index loaded: (2644, 4) records
✓ Historical Trading Data loaded: (211224, 16) records


In [9]:

# Convert date columns to datetime
fear_greed['date'] = pd.to_datetime(fear_greed['date'])
historical_data['Timestamp IST'] = pd.to_datetime(
    historical_data['Timestamp IST'], 
    format='%d-%m-%Y %H:%M'
)

# Create date column for merging
historical_data['date'] = historical_data['Timestamp IST'].dt.date
historical_data['date'] = pd.to_datetime(historical_data['date'])

# Extract time features
historical_data['hour'] = historical_data['Timestamp IST'].dt.hour
historical_data['day_of_week'] = historical_data['Timestamp IST'].dt.dayofweek
historical_data['month'] = historical_data['Timestamp IST'].dt.month
historical_data['year'] = historical_data['Timestamp IST'].dt.year

print(f"✓ Data preprocessed successfully")
print(f"  - Fear & Greed date range: {fear_greed['date'].min().date()} to {fear_greed['date'].max().date()}")
print(f"  - Trading data date range: {historical_data['date'].min().date()} to {historical_data['date'].max().date()}")

✓ Data preprocessed successfully
  - Fear & Greed date range: 2018-02-01 to 2025-05-02
  - Trading data date range: 2023-05-01 to 2025-05-01


In [11]:
print("\nExploratory Data Analysis...")

# 4.1: Market Sentiment Overview
print("\n--- Market Sentiment Distribution ---")
sentiment_dist = fear_greed['classification'].value_counts()
print(sentiment_dist)

# 4.2: Trading Activity Overview
print("\n--- Trading Activity Overview ---")
print(f"Total Trades: {len(historical_data):,}")
print(f"Unique Traders: {historical_data['Account'].nunique()}")
print(f"Unique Coins: {historical_data['Coin'].nunique()}")
print(f"Total Trading Volume (USD): ${historical_data['Size USD'].sum():,.2f}")
print(f"Average Trade Size (USD): ${historical_data['Size USD'].mean():,.2f}")

# 4.3: Side Distribution
print("\n--- Buy vs Sell Distribution ---")
side_dist = historical_data['Side'].value_counts()
print(side_dist)

# Save basic statistics
basic_stats = pd.DataFrame({
    'Metric': ['Total Trades', 'Unique Traders', 'Unique Coins', 
               'Total Volume USD', 'Avg Trade Size USD', 'Buy Trades', 'Sell Trades'],
    'Value': [len(historical_data), historical_data['Account'].nunique(),
              historical_data['Coin'].nunique(), historical_data['Size USD'].sum(),
              historical_data['Size USD'].mean(), side_dist.get('BUY', 0), 
              side_dist.get('SELL', 0)]
})
basic_stats.to_csv('csv_files/basic_statistics.csv', index=False)
print("✓ Basic statistics saved to csvfiles/basic_statistics.csv")


Exploratory Data Analysis...

--- Market Sentiment Distribution ---
classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64

--- Trading Activity Overview ---
Total Trades: 211,224
Unique Traders: 32
Unique Coins: 246
Total Trading Volume (USD): $1,191,187,442.46
Average Trade Size (USD): $5,639.45

--- Buy vs Sell Distribution ---
Side
SELL    108528
BUY     102696
Name: count, dtype: int64
✓ Basic statistics saved to csvfiles/basic_statistics.csv


In [13]:
print("\n[Step 4/8] Merging sentiment and trading data...")

# Merge trading data with sentiment
merged_data = historical_data.merge(
    fear_greed[['date', 'value', 'classification']], 
    on='date', 
    how='inner'
)

print(f"✓ Merged dataset created: {len(merged_data):,} records")
print(f"  - Date range: {merged_data['date'].min().date()} to {merged_data['date'].max().date()}")
print(f"  - Sentiment categories: {merged_data['classification'].unique()}")

# Save merged data
merged_data.to_csv('csv_files/merged_sentiment_trading.csv', index=False)
print("✓ Merged data saved to csvfiles/merged_sentiment_trading.csv")


[Step 4/8] Merging sentiment and trading data...
✓ Merged dataset created: 211,218 records
  - Date range: 2023-05-01 to 2025-05-01
  - Sentiment categories: ['Extreme Greed' 'Extreme Fear' 'Fear' 'Greed' 'Neutral']
✓ Merged data saved to csvfiles/merged_sentiment_trading.csv


In [15]:
print("\n[Step 5/8] Analyzing trader performance...")

# Calculate trader-level metrics
trader_metrics = merged_data.groupby('Account').agg({
    'Closed PnL': ['sum', 'mean', 'count'],
    'Size USD': ['sum', 'mean'],
    'Fee': 'sum',
    'Trade ID': 'count'
}).reset_index()

# Flatten column names
trader_metrics.columns = ['Account', 'Total_PnL', 'Avg_PnL_per_Trade', 'Num_Trades',
                          'Total_Volume_USD', 'Avg_Trade_Size_USD', 'Total_Fees',
                          'Trade_Count']

# Calculate net profit (PnL - Fees)
trader_metrics['Net_Profit'] = trader_metrics['Total_PnL'] - trader_metrics['Total_Fees']

# Classify traders by profitability
trader_metrics['Trader_Type'] = pd.cut(
    trader_metrics['Net_Profit'],
    bins=[-np.inf, 0, 1000, np.inf],
    labels=['Losing', 'Break-even', 'Profitable']
)

print(f"\n--- Trader Classification ---")
print(trader_metrics['Trader_Type'].value_counts())

# Save trader metrics
trader_metrics.to_csv('csv_files/trader_performance_metrics.csv', index=False)
print("✓ Trader metrics saved to csvfiles/trader_performance_metrics.csv")


[Step 5/8] Analyzing trader performance...

--- Trader Classification ---
Trader_Type
Profitable    28
Losing         4
Break-even     0
Name: count, dtype: int64
✓ Trader metrics saved to csvfiles/trader_performance_metrics.csv


In [16]:
print("\n[Step 6/8] Analyzing trading behavior by sentiment...")

# 7.1: Trading volume by sentiment
volume_by_sentiment = merged_data.groupby('classification').agg({
    'Size USD': ['sum', 'mean', 'count'],
    'Closed PnL': ['sum', 'mean']
}).reset_index()

volume_by_sentiment.columns = ['Sentiment', 'Total_Volume_USD', 'Avg_Trade_Size_USD',
                                'Num_Trades', 'Total_PnL', 'Avg_PnL']

print("\n--- Trading Metrics by Sentiment ---")
print(volume_by_sentiment)

# 7.2: Side preference by sentiment
side_by_sentiment = merged_data.groupby(['classification', 'Side']).size().unstack(fill_value=0)
side_by_sentiment['Buy_Sell_Ratio'] = side_by_sentiment['BUY'] / side_by_sentiment['SELL']

print("\n--- Buy/Sell Distribution by Sentiment ---")
print(side_by_sentiment)

# Save sentiment analysis
volume_by_sentiment.to_csv('csv_files/trading_by_sentiment.csv', index=False)
side_by_sentiment.to_csv('csv_files/side_preference_by_sentiment.csv')
print("✓ Sentiment analysis saved to csv_files/")

# 7.3: Profitability by sentiment
profitability_sentiment = merged_data.groupby('classification').agg({
    'Closed PnL': ['sum', 'mean', 'median', 'std'],
    'Fee': 'sum'
}).reset_index()

profitability_sentiment.columns = ['Sentiment', 'Total_PnL', 'Mean_PnL', 
                                    'Median_PnL', 'Std_PnL', 'Total_Fees']
profitability_sentiment['Net_PnL'] = (profitability_sentiment['Total_PnL'] - 
                                      profitability_sentiment['Total_Fees'])

print("\n--- Profitability by Sentiment ---")
print(profitability_sentiment)

profitability_sentiment.to_csv('csv_files/profitability_by_sentiment.csv', index=False)



[Step 6/8] Analyzing trading behavior by sentiment...

--- Trading Metrics by Sentiment ---
       Sentiment  Total_Volume_USD  Avg_Trade_Size_USD  Num_Trades  \
0   Extreme Fear      1.144843e+08         5349.731843       21400   
1  Extreme Greed      1.244652e+08         3112.251565       39992   
2           Fear      4.833248e+08         7816.109931       61837   
3          Greed      2.885825e+08         5736.884375       50303   
4        Neutral      1.802421e+08         4782.732661       37686   

      Total_PnL    Avg_PnL  
0  7.391102e+05  34.537862  
1  2.715171e+06  67.892861  
2  3.357155e+06  54.290400  
3  2.150129e+06  42.743559  
4  1.292921e+06  34.307718  

--- Buy/Sell Distribution by Sentiment ---
Side              BUY   SELL  Buy_Sell_Ratio
classification                              
Extreme Fear    10935  10465        1.044912
Extreme Greed   17940  22052        0.813532
Fear            30270  31567        0.958913
Greed           24576  25727        0.95526

In [17]:
print("\n[Step 7/8] Analyzing coin-level patterns...")

# Top coins by volume
top_coins = merged_data.groupby('Coin').agg({
    'Size USD': 'sum',
    'Trade ID': 'count',
    'Closed PnL': 'sum'
}).sort_values('Size USD', ascending=False).head(10)

top_coins.columns = ['Total_Volume_USD', 'Num_Trades', 'Total_PnL']
print("\n--- Top 10 Coins by Trading Volume ---")
print(top_coins)

# Coin performance by sentiment (top 5 coins only)
top_5_coins = top_coins.head(5).index.tolist()
coin_sentiment = merged_data[merged_data['Coin'].isin(top_5_coins)].groupby(
    ['Coin', 'classification']
).agg({
    'Size USD': 'sum',
    'Closed PnL': 'mean'
}).reset_index()

coin_sentiment.columns = ['Coin', 'Sentiment', 'Total_Volume_USD', 'Avg_PnL']

# Save coin analysis
top_coins.to_csv('csv_files/top_coins_analysis.csv')
coin_sentiment.to_csv('csv_files/coin_sentiment_analysis.csv', index=False)
print("✓ Coin analysis saved to csv_files/")



[Step 7/8] Analyzing coin-level patterns...

--- Top 10 Coins by Trading Volume ---
          Total_Volume_USD  Num_Trades     Total_PnL
Coin                                                
BTC           6.442321e+08       26064  8.680447e+05
HYPE          1.419902e+08       68005  1.948485e+06
SOL           1.250748e+08       10691  1.639556e+06
ETH           1.182810e+08       11158  1.319979e+06
@107          5.576086e+07       29992  2.783913e+06
FARTCOIN      8.311390e+06        4650 -1.006872e+05
SUI           7.781168e+06        1979  1.992688e+05
TRUMP         7.349347e+06        1920 -3.648249e+05
MELANIA       7.040710e+06        4428  3.903511e+05
XRP           5.343211e+06        1774  3.756901e+03
✓ Coin analysis saved to csv_files/


In [18]:
print("\n[Step 8/8] Generating advanced insights...")

# 9.1: Time-based patterns
hourly_activity = merged_data.groupby(['hour', 'classification']).agg({
    'Size USD': 'sum',
    'Trade ID': 'count'
}).reset_index()
hourly_activity.columns = ['Hour', 'Sentiment', 'Total_Volume', 'Num_Trades']
hourly_activity.to_csv('csv_files/hourly_trading_patterns.csv', index=False)

# 9.2: Risk analysis (using trade size as proxy)
merged_data['Risk_Category'] = pd.cut(
    merged_data['Size USD'],
    bins=[0, 100, 1000, 10000, np.inf],
    labels=['Low', 'Medium', 'High', 'Very High']
)

risk_sentiment = merged_data.groupby(['Risk_Category', 'classification']).agg({
    'Trade ID': 'count',
    'Closed PnL': 'mean'
}).reset_index()
risk_sentiment.columns = ['Risk_Category', 'Sentiment', 'Num_Trades', 'Avg_PnL']
risk_sentiment.to_csv('csv_files/risk_analysis_by_sentiment.csv', index=False)

# 9.3: Winning vs Losing trades by sentiment
merged_data['Trade_Outcome'] = merged_data['Closed PnL'].apply(
    lambda x: 'Win' if x > 0 else ('Loss' if x < 0 else 'Break-even')
)

outcome_sentiment = merged_data.groupby(['classification', 'Trade_Outcome']).size().unstack(fill_value=0)
outcome_sentiment['Win_Rate'] = (outcome_sentiment['Win'] / 
                                 (outcome_sentiment['Win'] + outcome_sentiment['Loss'])) * 100
outcome_sentiment.to_csv('csv_files/trade_outcomes_by_sentiment.csv')

print("✓ Advanced insights saved to csvfiles/")


[Step 8/8] Generating advanced insights...
✓ Advanced insights saved to csvfiles/


In [20]:
print("\n" + "=" * 80)
print("KEY FINDINGS SUMMARY")
print("=" * 80)

# Finding 1: Best sentiment for profitability
best_sentiment = profitability_sentiment.loc[profitability_sentiment['Net_PnL'].idxmax(), 'Sentiment']
best_pnl = profitability_sentiment.loc[profitability_sentiment['Net_PnL'].idxmax(), 'Net_PnL']

print(f"\n1. MOST PROFITABLE SENTIMENT: {best_sentiment}")
print(f"   Net Profit: ${best_pnl:,.2f}")

# Finding 2: Trading volume patterns
highest_vol_sentiment = volume_by_sentiment.loc[volume_by_sentiment['Total_Volume_USD'].idxmax(), 'Sentiment']
print(f"\n2. HIGHEST TRADING VOLUME SENTIMENT: {highest_vol_sentiment}")

# Finding 3: Risk behavior
print(f"\n3. RISK-TAKING BEHAVIOR:")
high_risk_trades = len(merged_data[merged_data['Risk_Category'] == 'Very High'])
print(f"   Very High Risk Trades: {high_risk_trades:,} ({high_risk_trades/len(merged_data)*100:.1f}%)")

# Finding 4: Top performing traders
top_3_traders = trader_metrics.nlargest(3, 'Net_Profit')
print(f"\n4. TOP 3 TRADERS BY NET PROFIT:")
for idx, row in top_3_traders.iterrows():
    print(f"   - {row['Account'][:10]}...: ${row['Net_Profit']:,.2f}")

# Finding 5: Most traded coin (pick top entry and format count properly)
most_traded_counts = merged_data['Coin'].value_counts()
if len(most_traded_counts) > 0:
    most_traded_coin = most_traded_counts.index[0]
    most_traded_count = int(most_traded_counts.iloc[0])
    print(f"\n5. MOST TRADED ASSET: {most_traded_coin}")
    print(f"   Total Trades: {most_traded_count:,}")
else:
    print("\n5. MOST TRADED ASSET: None (no trades found)")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE!")
print("=" * 80)
print("\nAll outputs saved to:")
print("  - csvfiles/ : All data exports and analysis results")
print("  - outputs/  : Charts and visualizations (run visualization section)")
print("\n" + "=" * 80)



KEY FINDINGS SUMMARY

1. MOST PROFITABLE SENTIMENT: Fear
   Net Profit: $3,264,698.49

2. HIGHEST TRADING VOLUME SENTIMENT: Fear

3. RISK-TAKING BEHAVIOR:
   Very High Risk Trades: 18,687 (8.8%)

4. TOP 3 TRADERS BY NET PROFIT:
   - 0xb1231a4a...: $2,127,387.28
   - 0x083384f8...: $1,592,824.51
   - 0xbaaaf657...: $931,567.10

5. MOST TRADED ASSET: HYPE
   Total Trades: 68,005

ANALYSIS COMPLETE!

All outputs saved to:
  - csvfiles/ : All data exports and analysis results
  - outputs/  : Charts and visualizations (run visualization section)



In [21]:
print("\n[Visualization] Creating charts...")

# Chart 1: Sentiment Distribution
fig, ax = plt.subplots(figsize=(10, 6))
sentiment_counts = fear_greed['classification'].value_counts()
colors = ['#d32f2f', '#f57c00', '#fbc02d', '#7cb342', '#388e3c']
ax.bar(sentiment_counts.index, sentiment_counts.values, color=colors, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Sentiment Classification', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Days', fontsize=12, fontweight='bold')
ax.set_title('Bitcoin Market Sentiment Distribution (Fear & Greed Index)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 1 saved: sentiment_distribution.png")

# Chart 2: Trading Volume by Sentiment
fig, ax = plt.subplots(figsize=(10, 6))
volume_by_sentiment_sorted = volume_by_sentiment.sort_values('Total_Volume_USD', ascending=False)
bars = ax.bar(volume_by_sentiment_sorted['Sentiment'], 
               volume_by_sentiment_sorted['Total_Volume_USD']/1e6,
               color=['#388e3c', '#7cb342', '#fbc02d', '#f57c00', '#d32f2f'],
               edgecolor='black', linewidth=1.5)
ax.set_xlabel('Sentiment', fontsize=12, fontweight='bold')
ax.set_ylabel('Trading Volume (Million USD)', fontsize=12, fontweight='bold')
ax.set_title('Total Trading Volume by Market Sentiment', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:.1f}M',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/volume_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 2 saved: volume_by_sentiment.png")

# Chart 3: Profitability by Sentiment
fig, ax = plt.subplots(figsize=(10, 6))
profit_sorted = profitability_sentiment.sort_values('Net_PnL', ascending=False)
colors_profit = ['green' if x > 0 else 'red' for x in profit_sorted['Net_PnL']]
bars = ax.bar(profit_sorted['Sentiment'], profit_sorted['Net_PnL'],
               color=colors_profit, edgecolor='black', linewidth=1.5, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Sentiment', fontsize=12, fontweight='bold')
ax.set_ylabel('Net Profit/Loss (USD)', fontsize=12, fontweight='bold')
ax.set_title('Net Profitability by Market Sentiment (PnL - Fees)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    label_y = height if height > 0 else height - 500
    ax.text(bar.get_x() + bar.get_width()/2., label_y,
            f'${height:,.0f}',
            ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=10, fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/profitability_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 3 saved: profitability_by_sentiment.png")

# Chart 4: Buy vs Sell Ratio by Sentiment
fig, ax = plt.subplots(figsize=(10, 6))
side_by_sentiment_sorted = side_by_sentiment.sort_values('Buy_Sell_Ratio', ascending=False)
ax.bar(side_by_sentiment_sorted.index, side_by_sentiment_sorted['Buy_Sell_Ratio'],
       color='steelblue', edgecolor='black', linewidth=1.5)
ax.axhline(y=1, color='red', linestyle='--', linewidth=2, label='Equal Buy/Sell')
ax.set_xlabel('Sentiment', fontsize=12, fontweight='bold')
ax.set_ylabel('Buy/Sell Ratio', fontsize=12, fontweight='bold')
ax.set_title('Buy/Sell Trading Ratio by Market Sentiment', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels
for i, (idx, val) in enumerate(side_by_sentiment_sorted['Buy_Sell_Ratio'].items()):
    ax.text(i, val + 0.02, f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/buy_sell_ratio_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 4 saved: buy_sell_ratio_by_sentiment.png")

# Chart 5: Top 10 Coins Trading Volume
fig, ax = plt.subplots(figsize=(12, 6))
top_10_coins = merged_data.groupby('Coin')['Size USD'].sum().sort_values(ascending=False).head(10)
ax.barh(range(len(top_10_coins)), top_10_coins.values/1e6, color='coral', edgecolor='black', linewidth=1.5)
ax.set_yticks(range(len(top_10_coins)))
ax.set_yticklabels(top_10_coins.index)
ax.set_xlabel('Trading Volume (Million USD)', fontsize=12, fontweight='bold')
ax.set_ylabel('Coin', fontsize=12, fontweight='bold')
ax.set_title('Top 10 Most Traded Coins by Volume', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, val in enumerate(top_10_coins.values/1e6):
    ax.text(val + 0.5, i, f'${val:.1f}M', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/top_10_coins_volume.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 5 saved: top_10_coins_volume.png")

# Chart 6: Trader Performance Distribution
fig, ax = plt.subplots(figsize=(10, 6))
trader_type_counts = trader_metrics['Trader_Type'].value_counts()
colors_trader = ['#d32f2f', '#fbc02d', '#388e3c']
explode = (0.05, 0.05, 0.05)
ax.pie(trader_type_counts.values, labels=trader_type_counts.index, autopct='%1.1f%%',
       colors=colors_trader, explode=explode, shadow=True, startangle=90,
       textprops={'fontsize': 12, 'fontweight': 'bold'})
ax.set_title('Trader Classification by Profitability', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('outputs/trader_classification.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 6 saved: trader_classification.png")

# Chart 7: Hourly Trading Activity Heatmap
hourly_pivot = hourly_activity.pivot(index='Sentiment', columns='Hour', values='Total_Volume')
fig, ax = plt.subplots(figsize=(14, 6))
sns.heatmap(hourly_pivot, cmap='YlOrRd', annot=False, fmt='.0f', cbar_kws={'label': 'Volume (USD)'},
            linewidths=0.5, ax=ax)
ax.set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
ax.set_ylabel('Sentiment', fontsize=12, fontweight='bold')
ax.set_title('Trading Volume Heatmap: Sentiment vs Hour of Day', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/hourly_activity_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 7 saved: hourly_activity_heatmap.png")

# Chart 8: Risk Analysis by Sentiment
risk_counts = merged_data.groupby(['classification', 'Risk_Category']).size().unstack(fill_value=0)
risk_counts_pct = risk_counts.div(risk_counts.sum(axis=1), axis=0) * 100

fig, ax = plt.subplots(figsize=(10, 6))
risk_counts_pct.plot(kind='bar', stacked=True, ax=ax, 
                      color=['#4caf50', '#ffeb3b', '#ff9800', '#f44336'],
                      edgecolor='black', linewidth=1)
ax.set_xlabel('Sentiment', fontsize=12, fontweight='bold')
ax.set_ylabel('Percentage of Trades (%)', fontsize=12, fontweight='bold')
ax.set_title('Risk Profile Distribution by Market Sentiment', fontsize=14, fontweight='bold')
ax.legend(title='Risk Category', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/risk_distribution_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Chart 8 saved: risk_distribution_by_sentiment.png")

print("\n" + "=" * 80)
print("ALL VISUALIZATIONS COMPLETE!")
print("=" * 80)
print(f"\nTotal charts created: 8")
print("All charts saved in outputs/ directory")


[Visualization] Creating charts...
✓ Chart 1 saved: sentiment_distribution.png
✓ Chart 2 saved: volume_by_sentiment.png
✓ Chart 3 saved: profitability_by_sentiment.png
✓ Chart 4 saved: buy_sell_ratio_by_sentiment.png
✓ Chart 5 saved: top_10_coins_volume.png
✓ Chart 6 saved: trader_classification.png
✓ Chart 7 saved: hourly_activity_heatmap.png
✓ Chart 8 saved: risk_distribution_by_sentiment.png

ALL VISUALIZATIONS COMPLETE!

Total charts created: 8
All charts saved in outputs/ directory
